In [ ]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from torchvision import models, transforms
from PIL import Image
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch

In [ ]:
model = models.alexnet(weights="IMAGENET1K_V1")
model.eval() # Set the model to evaluation mode

# Define the transformation space
space = [
    Integer(-10, 10, name='rotation'),  # Range of rotation angles
    Real(0.1, 0.9, name='darken'),  # Range of darkening factors
    Real(0.5, 1.5, name='saturation')  # Range of saturation factors
]

In [ ]:

dataset_path = '../Dataset/imagenet-mini/train'
dataset = ImageFolder(root=dataset_path, transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
]))
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


In [ ]:
def objective(params):
    rotation, darken, saturation = params

    # Select a random image from the dataset
    img, _ = next(iter(dataloader))
    img = transforms.ToPILImage()(img.squeeze(0))

    # Apply custom transformations
    img = img.rotate(rotation)  # Rotate the image
    img = transforms.functional.adjust_brightness(img, darken)  # Darken the image
    img = transforms.functional.adjust_saturation(img, saturation)  # Adjust saturation

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = transform(img)  # Apply the standard pre-processing

    # Add batch dimension and send to model
    img = img.unsqueeze(0)
    output = model(img)
    
    # Calculate failure metric
    
    failure_metric = -output.max().item()  # Negative sign because we're minimizing

    print(failure_metric)
    
    return failure_metric

In [ ]:
# Run Bayesian Optimization
res_gp = gp_minimize(objective, space, n_calls=500, random_state=0)

# Results
print("Best parameters: {}".format(res_gp.x))
print("Best objective: {}".format(res_gp.fun))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Objective function values over iterations
plt.figure(figsize=(12, 6))
plt.plot(res_gp.func_vals)
plt.xlabel('Iteration')
plt.ylabel('Objective Function Value')
plt.title('Model Failure Metric Over Iterations')
plt.grid(True)
plt.show()

# Parameter values over iterations
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
parameters = np.array(res_gp.x_iters)
objective_values = res_gp.func_vals

# Rotation
axs[0].scatter(range(len(parameters)), parameters[:, 0], c=objective_values, cmap='viridis')
axs[0].set_title('Rotation Angle Over Iterations')
axs[0].set_xlabel('Iteration')
axs[0].set_ylabel('Rotation Angle')

# Darken
axs[1].scatter(range(len(parameters)), parameters[:, 1], c=objective_values, cmap='viridis')
axs[1].set_title('Darken Factor Over Iterations')
axs[1].set_xlabel('Iteration')
axs[1].set_ylabel('Darken Factor')

# Saturation
axs[2].scatter(range(len(parameters)), parameters[:, 2], c=objective_values, cmap='viridis')
axs[2].set_title('Saturation Factor Over Iterations')
axs[2].set_xlabel('Iteration')
axs[2].set_ylabel('Saturation Factor')

for ax in axs:
    ax.grid(True)

plt.colorbar(axs[2].scatter(range(len(parameters)), parameters[:, 2], c=objective_values, cmap='viridis'), ax=axs, orientation='vertical', label='Objective Function Value')
plt.show()
